In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime 

import sqlite3
import sys
import time
import tqdm
from tqdm.auto import tqdm
import pickle
import joblib
import os

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'


## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [17]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = + data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [18]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

In [19]:
# # Save the dictionary of DataFrames as a pickle
# with open(data_path + 'dfs_dict.pkl', 'wb') as f:
#     pickle.dump(dfs, f)

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [ ]:
players_df = dfs['players_df']
players_df.head()


In [ ]:
ranking_df = dfs['ranking_df']
ranking_df.head()

In [ ]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

In [ ]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data")
sets_df.shape



In [ ]:
tournament_info_df = dfs['tournament_info_df']
print(tournament_info_df.shape)
tournament_info_df.head()


## Overall Glicko-2 Exploration ##


Import weekly updated Glicko-2 rating.

In [ ]:
player_ratings_df = pd.read_pickle(data_path + 'overall_players_ranking_new_weekly.pkl')
print(player_ratings_df.shape)
player_ratings_df.head()

## Number of Glicko-2 updates
Running total of number of updates to each players glicko-2 rating. We use numba njit and prange to speed up the loops in the function. We save the results so that we only need to run the calculation once.

In [ ]:

def previous_updates(array):
    """ This funcion returns an array like array with the number of times the value above i,j entry of array has changed.
    Args:
        array (np): the array

    Returns:
        np: the number of times array has changed above the i,j entry
    """
    previous_updates = np.zeros_like(array, dtype=np.int32)
    
    for i in range(1, array.shape[0]-1): # row i
        previous_row = array[i-1,:]
        # print(previous_row)
        current_row = array[i,:]
        # print(current_row)
        change = (previous_row != current_row).astype(np.int32)
        change

        previous_updates[i+1,:] = previous_updates[i,:] + change

    return previous_updates

## Testing array
# array = np.array([
#     [1, 1, 1],
#     [1, 1, 2],
#     [1, 2, 3],
#     [1, 3, 4]])

# print(array)
# previous_updates(array)
# print(previous_updates(array))

# # Do the calculation once.
# player_ratings_np = player_ratings_df.to_numpy()
# start = time.time()
# number_of_rating_updates_df = pd.DataFrame(columns=player_ratings_df.columns, index=player_ratings_df.index, data=previous_updates(player_ratings_np))
# end = time.time()
# print(f'time = {end-start:.2f}')
# number_of_rating_updates_df.head()

# # Save the results
# number_of_rating_updates_df.to_pickle(data_path + 'number_of_rating_updates_df.pkl')

## Load the results
number_of_rating_updates_df = pd.read_pickle(data_path + 'number_of_rating_updates_df.pkl')
number_of_rating_updates_df.head()

## Add some columns to sets_df
We add the start of the tournament, the player ratings at the start of the tournament, and the number of times the player's rating has been updated before the start of the tournament.

In [27]:
# Perform a merge on 'key' and 'tournament_key' to bring 'start' dates into sets_df
merged_df = sets_df.merge(tournament_info_df[['key', 'start']], left_on='tournament_key', right_on='key', how='left')


In [ ]:
tqdm.pandas()
import swifter

# Function to get both Player 1 and Player 2 ratings and the number of rating updates
def get_ratings_and_updates(row, player_ratings_df, number_of_rating_updates_df):
    # Find the closest date in player_ratings_df that is <= 'start' date
    closest_date = player_ratings_df.index[player_ratings_df.index <= row['start']].max()
    
    # If there's no valid date, return None for ratings and updates
    if pd.isnull(closest_date):
        return pd.Series([None, None, None, None], index=['p1_rating', 'p2_rating', 'p1_updates', 'p2_updates'])
    
    # Fetch Player 1's and Player 2's ratings on the closest date
    p1_rating = player_ratings_df.loc[closest_date, row['p1_id']] if row['p1_id'] in player_ratings_df.columns else None
    p2_rating = player_ratings_df.loc[closest_date, row['p2_id']] if row['p2_id'] in player_ratings_df.columns else None
    
    # Fetch Player 1's and Player 2's number of rating updates on the closest date
    p1_updates = number_of_rating_updates_df.loc[closest_date, row['p1_id']] if row['p1_id'] in number_of_rating_updates_df.columns else None
    p2_updates = number_of_rating_updates_df.loc[closest_date, row['p2_id']] if row['p2_id'] in number_of_rating_updates_df.columns else None
    
    # Return all values as a pandas Series
    return pd.Series([p1_rating, p2_rating, p1_updates, p2_updates], 
                     index=['p1_rating', 'p2_rating', 'p1_updates', 'p2_updates'])

# Apply the function to each row in merged_df
# merged_df[['p1_rating', 'p2_rating', 'p1_updates', 'p2_updates']] = merged_df.progress_apply(
#     get_ratings_and_updates, axis=1, 
#     player_ratings_df=player_ratings_df, 
#     number_of_rating_updates_df=number_of_rating_updates_df,
# )

# Save
# merged_df.to_pickle(data_path + 'augmented_sets_df.pkl')

# # Load
augmented_sets_df = pd.read_pickle(data_path + 'tournament_sets_with_top_8_df.pkl')
augmented_sets_df.head()

## Top 8 Locations
We look for tournaments with a sets with 'location_names' = ['GF', 'Grand Final', 'Grand Final'].
The assumption is that tournaments with that location name will have a consistent location name structure.

In [ ]:

# # Filter the rows where 'location_names' exactly matches ['GF', 'Grand Final', 'Grand Final']
# gf_sets_df = sets_df[augmented_sets_df['location_names'].apply(lambda x: x == ['GF', 'Grand Final', 'Grand Final'])]

# # Extract the tournament keys for the Grand Finals
# gf_tournament_keys = list(gf_sets_df['tournament_key'])

# Filter the rows where 'location_names' exactly matches ['GF', 'Grand Final', 'Grand Final']
gf_sets_df = augmented_sets_df[augmented_sets_df['location_names'].apply(lambda x: x == ['GF', 'Grand Final', 'Grand Final'])]

# Extract the tournament keys for the Grand Finals
gf_tournament_keys = list(gf_sets_df['tournament_key'])

# Filter the sets_df to include only the sets from tournaments that had Grand Finals
valid_tournament_sets_df = augmented_sets_df[augmented_sets_df['tournament_key'].isin(gf_tournament_keys)]

# Display the result
print(valid_tournament_sets_df.head(3))
print(valid_tournament_sets_df['location_names'].value_counts().to_string())

The location names of the top 8 games are the following:
- [f"L{n}", f"Losers {n}", f"Losers Round {n}], # Where n is the maximum n in all such location of the  tournament.  
- ['WSF', 'Winners Semis', 'Winners Semi-Final'],
- ['LQF', 'Losers Quarters', 'Losers Quarter-Final'],
- ['WF', 'Winners Final', 'Winners Final'],
- ['LSF', 'Losers Semis', 'Losers Semi-Final'],
- ['LF', 'Losers Final', 'Losers Final'],
- ['GF', 'Grand Final', 'Grand Final'],
- ['GFR', 'GF Reset', 'Grand Final Reset']

In [30]:
# The vast majority of the top 8 games have these "location_names"
top_8_locations = [                                   
        ['WSF', 'Winners Semis', 'Winners Semi-Final'],
        ['LQF', 'Losers Quarters', 'Losers Quarter-Final'],
        ['WF', 'Winners Final', 'Winners Final'],
        ['LSF', 'Losers Semis', 'Losers Semi-Final'],
        ['LF', 'Losers Final', 'Losers Final'],
        ['GF', 'Grand Final', 'Grand Final'],
        ['GFR', 'GF Reset', 'Grand Final Reset']
    ] 

In [ ]:
def take_first_item(location_list):
    return location_list[0]
valid_tournament_sets_df['location_names'] = valid_tournament_sets_df['location_names'].progress_apply(take_first_item)


In [32]:
# import re



# def max_losers_round_pattern(location_name):
#     losers_round_pattern = r'^L(\d+)$'
#     return re.match(losers_round_pattern, location_name)
        
# def find_last_losers(tournament_key):
#     tournament_sets_df = valid_tournament_sets_df[valid_tournament_sets_df['tournament_key'] == tournament_key]
    
#     mask = tournament_sets_df['location_names'].apply(max_losers_round_pattern)
    
#     losers_rounds = tournament_sets_df[mask]['location_names'][1:].to(int)
#     return max(losers_rounds)



# valid_tournaments_df = tournament_info_df[tournament_info_df['key'].isin(gf_tournament_keys)]

# valid_tournaments_df['last_losers_round'] = valid_tournaments_df['key'].progress_apply(find_last_losers)

    

In [ ]:
import re
# import pandas as pd

# Compile the regex pattern once
losers_round_pattern = re.compile(r'^L(\d+)$')

# Function to extract the first item from the list in 'location_names'
def take_first_item(location_list):
    return location_list[0]

# Function to extract the round number 'n' from location names like 'L{n}', returns None if no match
def extract_losers_round(location_name):
    match = losers_round_pattern.match(location_name)
    if match:
        return int(match.group(1))
    return None

# Extract the losers round numbers in one go
valid_tournament_sets_df['losers_round_n'] = valid_tournament_sets_df['location_names'].apply(extract_losers_round)

# Group by 'tournament_key' and find the maximum losers round for each tournament
max_losers_round_by_tournament = valid_tournament_sets_df.groupby('tournament_key')['losers_round_n'].max().reset_index()

# Filter tournaments based on 'gf_tournament_keys'
valid_tournaments_df = tournament_info_df[tournament_info_df['key'].isin(gf_tournament_keys)]

# Merge the max losers round with the tournaments DataFrame
valid_tournaments_df = valid_tournaments_df.merge(max_losers_round_by_tournament, left_on='key', right_on='tournament_key', how='left')

# Rename the column for clarity
valid_tournaments_df.rename(columns={'losers_round_n': 'last_losers_round'}, inplace=True)

# Fill NaN values with a default value (e.g., -1 or 0, depending on your requirements)
valid_tournaments_df['last_losers_round'].fillna(0, inplace=True)

# Convert the column to integer type
valid_tournaments_df['last_losers_round'] = valid_tournaments_df['last_losers_round'].astype(int)

# Display the result
print(valid_tournaments_df['last_losers_round'])


In [ ]:
top_8_locations = ['WSF']

# Create a dictionary to map each tournament_key to its last_losers_round for faster lookups
last_losers_round_dict = valid_tournaments_df.set_index('key')['last_losers_round'].to_dict()

# Vectorized check if 'location_names' is in the top 8 predefined locations
top_8_mask = valid_tournament_sets_df['location_names'].isin(top_8_locations)

# Vectorized check if the 'location_names' is equal to the corresponding 'last_losers_round' for each tournament_key
losers_round_mask = valid_tournament_sets_df.apply(
    lambda row: row['location_names'] == f"L{last_losers_round_dict.get(row['tournament_key'], '')}",
    axis=1
)

# Combine the two conditions using bitwise OR (|) to get the final mask
top_8_combined_mask = top_8_mask | losers_round_mask

# Filter the DataFrame based on the combined mask
valid_top_8_sets_df = valid_tournament_sets_df[top_8_combined_mask]

# Display the result
valid_top_8_sets_df.head()


In [ ]:
valid_top_8_sets_df.head(20)

In [ ]:
# Step 1: Group by 'tournament_key' and check if any of the players in top 8 have updates == 0
tournaments_with_zero_updates = valid_top_8_sets_df.groupby('tournament_key').filter(
    lambda x: not ((x['p1_updates'] <= 1).any() or (x['p2_updates'] <= 1).any())
)
tournaments_with_zero_updates['higher_rated_won'].sum()/tournaments_with_zero_updates.shape[0]
# Step 2: Display the filtered DataFrame
# tournaments_with_zero_updates.head(20)


In [ ]:
tournaments_with_zero_updates['higher_rated_won'].sum()/tournaments_with_zero_updates.shape[0]

In [ ]:
tournaments_with_zero_updates.shape[0]